Machine Learning Series: Image Classification dengan Convolutional Neural Network (CNN)


Classification Gunting Batu Kertas

In [ ]:
# Mengimpor pustaka/modul python yang dibutuhkan
import zipfile
import os
import glob 
import warnings

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

from keras.preprocessing import image
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

In [ ]:
!kaggle datasets download -d drgfreeman/rockpaperscissors

In [ ]:
!mkdir datasets
!unzip -qq rockpaperscissors.zip -d dataset
!ls dataset

mkdir: cannot create directory ‘datasets’: File exists
10.jpeg  18.jpeg  25.jpeg  33.jpeg  40.jpeg  README_rpc-cv-images.txt
11.jpeg  19.jpeg  26.jpeg  34.jpeg  4.jpeg   rock
12.jpeg  1.jpeg   27.jpeg  35.jpeg  5.jpeg   rps-cv-images
13.jpeg  20.jpeg  28.jpeg  36.jpeg  6.jpeg   scissors
14.jpeg  21.jpeg  2.jpeg   37.jpeg  7.jpeg
15.jpeg  22.jpeg  30.jpeg  38.jpeg  8.jpeg
16.jpeg  23.jpeg  31.jpeg  39.jpeg  9.jpeg
17.jpeg  24.jpeg  32.jpeg  3.jpeg   paper


In [ ]:
import os

dataset = os.path.join('/content/dataset')

print(os.listdir(dataset))

['14.jpeg', 'scissors', '27.jpeg', '21.jpeg', '19.jpeg', '6.jpeg', '34.jpeg', '11.jpeg', '15.jpeg', '4.jpeg', 'rock', '24.jpeg', '28.jpeg', '9.jpeg', '30.jpeg', '5.jpeg', '23.jpeg', '20.jpeg', '22.jpeg', '39.jpeg', '7.jpeg', '8.jpeg', '38.jpeg', '17.jpeg', '16.jpeg', '40.jpeg', '25.jpeg', '2.jpeg', '3.jpeg', 'README_rpc-cv-images.txt', '36.jpeg', 'paper', '26.jpeg', '35.jpeg', '18.jpeg', '37.jpeg', '1.jpeg', '10.jpeg', '32.jpeg', '33.jpeg', '13.jpeg', '12.jpeg', '31.jpeg', 'rps-cv-images']


In [ ]:
data = ['paper', 'scissors', 'rock']

for data in data:
    print(f'{data} images: ', len(os.listdir(f'/content/dataset/{data}')))

    total_image = len(list(glob.iglob("/content/dataset/*/*.*", recursive=True)))
print("Total Data Image JPEG     : ",total_image)

paper images:  712
scissors images:  750
rock images:  726
Total Data Image JPEG     :  2189


In [ ]:
import shutil
base_dir = '/content/dataset/'
shutil.rmtree(os.path.join(base_dir, 'rps-cv-images'))
os.remove(os.path.join(base_dir, 'README_rpc-cv-images.txt'))

base_dir = '/content/dataset/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [ ]:
datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=0.4,
                    shear_range = 0.2,
                    brightness_range = [0,1],
                    horizontal_flip=True,
                    vertical_flip= True,
                    fill_mode = 'nearest',
                    validation_split = 0.2) #membagi validation set 20%


In [ ]:
train_generator = datagen.flow_from_directory(                           
	        base_dir,                                                                      
	        target_size=(150,150),                                                          
	        class_mode='categorical',                                                       
          shuffle=True,                                                                   
          subset='training'                                                              
          )

validation_generator = datagen.flow_from_directory(                      
	      base_dir,                                                                     
	      target_size=(150,150),                                                          
	      class_mode='categorical',                                                      
        shuffle=True,
        subset='validation'                                                            
        )

Found 1751 images belonging to 3 classes.
Found 437 images belonging to 3 classes.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3), activation= 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(128,(3,3), activation= 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(256,(3,3), activation= 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation= 'relu'),
  tf.keras.layers.Dropout(0.1),                                                  #menambahkan dropout
  tf.keras.layers.Dense(3, activation= 'softmax')
])

model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

In [ ]:
def scheduler(epoch, lr):
  if epoch < 5:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs', histogram_freq=0, write_graph=True, write_images=False,
    update_freq='epoch', embeddings_freq=0,
    embeddings_metadata=None
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 25,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 5,
    verbose =2
)

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():

  path = fn 
  img = image.load_img(path, target_size =(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)

  print(fn)
  if classes[0,0]!=0:
    print('paper')
  elif classes[0,1]!=0:
    print('rock')
  else:
    print('scissors')